##### Direct DL

In [ ]:
import os
import json
import warnings
import joblib
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List
from itertools import combinations
from functools import partial

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix,
    log_loss, roc_curve, precision_recall_curve, auc
)
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from scipy.optimize import minimize

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

HAS_XGB = False
HAS_LGBM = False
HAS_CAT = False
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    pass
try:
    import lightgbm as lgb
    HAS_LGBM = True
except ImportError:
    pass
try:
    from catboost import CatBoostClassifier
    HAS_CAT = True
except ImportError:
    pass

warnings.filterwarnings("ignore", category=UserWarning)


CSV_PATH = "/kaggle/working/features_256d_efficientnet.csv"
ROOT_DIR = "/kaggle/working/Customized CNN"
OUT_DIR  = "/kaggle/working/Ensemble Model All Version/Version3"
os.makedirs(OUT_DIR, exist_ok=True)

SAVED_MODELS = {
    "svm": f"{ROOT_DIR}/SVM/svm_final_trainval.joblib",
    "xgb": f"{ROOT_DIR}/XGB/xgb_final_trainval.joblib",
    "rf" : f"{ROOT_DIR}/RF/rf_final_trainval.joblib",
    "knn": f"{ROOT_DIR}/KNN/knn_final_trainval.joblib",
    "cat": f"{ROOT_DIR}/CAT/cat_final_trainval.cbm",
}

rng = 42


def setup_plot_style():
    plt.rcParams.update({
        "font.family": "Times New Roman", "font.size": 12, "axes.labelsize": 16,
        "axes.titlesize": 18, "font.weight": "bold", "axes.labelweight": "bold"
    })

def plot_confusion_matrix(cm: np.ndarray, classes: List[str], output_path: str, title: str):
    setup_plot_style()
    cm_df = pd.DataFrame(cm, index=classes, columns=classes)
    def wrap(lbl):
        p = str(lbl).split(); return lbl if len(p) <= 1 else p[0] + "\n" + " ".join(p[1:])
    labels = [wrap(c) for c in cm_df.columns]
    cmap_teal = LinearSegmentedColormap.from_list("tealgrad", ["#d9f0f3", "#007c7c"], N=256)
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(
        cm_df, annot=True, fmt="d", cmap=cmap_teal, cbar=True,
        xticklabels=labels, yticklabels=labels, linewidths=1,
        linecolor="white", annot_kws={"fontsize": 14, "weight": "bold"}, ax=ax
    )
    ax.set_title(title, weight="bold")
    ax.set_xlabel("Predicted", weight="bold"); ax.set_ylabel("Actual", weight="bold")
    for label in ax.get_xticklabels() + ax.get_yticklabels(): label.set_fontweight("bold")
    fig.tight_layout()
    fig.savefig(output_path, dpi=600, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved confusion matrix to {output_path}")

def plot_roc_pr_curves(y_true_bin: np.ndarray, y_pred_proba: np.ndarray, classes: List[str], out_dir: str, prefix: str):
    setup_plot_style()
    n_classes = len(classes)

    fig_roc, ax_roc = plt.subplots(figsize=(10, 8))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        ax_roc.plot(fpr, tpr, lw=2, label=f'{classes[i]} (AUC = {roc_auc:0.2f})')
    ax_roc.plot([0, 1], [0, 1], 'k--', lw=2)
    ax_roc.set_xlim([0.0, 1.0]); ax_roc.set_ylim([0.0, 1.05])
    ax_roc.set_xlabel('False Positive Rate'); ax_roc.set_ylabel('True Positive Rate')
    ax_roc.set_title(f'Receiver Operating Characteristic (ROC) - {prefix}')
    ax_roc.legend(loc="lower right", fontsize=10)
    fig_roc.tight_layout()
    roc_path = os.path.join(out_dir, f"roc_curve_{prefix}.png")
    fig_roc.savefig(roc_path, dpi=600, bbox_inches="tight")
    plt.close(fig_roc)
    print(f"Saved ROC curve to {roc_path}")

    fig_pr, ax_pr = plt.subplots(figsize=(10, 8))
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_pred_proba[:, i])
        ax_pr.plot(recall, precision, lw=2, label=f'{classes[i]}')
    ax_pr.set_xlim([0.0, 1.0]); ax_pr.set_ylim([0.0, 1.05])
    ax_pr.set_xlabel('Recall'); ax_pr.set_ylabel('Precision')
    ax_pr.set_title(f'Precision-Recall Curve - {prefix}')
    ax_pr.legend(loc="best", fontsize=10)
    fig_pr.tight_layout()
    pr_path = os.path.join(out_dir, f"pr_curve_{prefix}.png")
    fig_pr.savefig(pr_path, dpi=600, bbox_inches="tight")
    plt.close(fig_pr)
    print(f"Saved PR curve to {pr_path}")

df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
if not feat_cols: raise RuntimeError("No feature columns (f0, f1, ...) found.")
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(int)
classes = df.sort_values("class_idx")["label"].unique().tolist()
n_classes = len(classes)

X_trval, X_test, y_trval, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=rng)
print("Split sizes:", {"trainval": len(y_trval), "test": len(y_test)})


def softmax(z: np.ndarray) -> np.ndarray:
    z = z - np.max(z, axis=1, keepdims=True)
    ez = np.exp(z)
    return ez / np.sum(ez, axis=1, keepdims=True)

def predict_logits(model, Xs) -> np.ndarray:
    if hasattr(model, "decision_function"):
        d = model.decision_function(Xs)
        if d.ndim == 1: d = np.vstack([-d, d]).T
        return np.asarray(d, dtype=np.float64)
    elif hasattr(model, "predict_proba"):
        return np.log(np.clip(model.predict_proba(Xs), 1e-12, 1.0))
    else:
        pred = model.predict(Xs)
        L = np.full((Xs.shape[0], n_classes), -10.0, dtype=np.float64)
        L[np.arange(Xs.shape[0]), pred] = 10.0
        return L

def predict_proba_safe(model, Xs) -> np.ndarray:
    if hasattr(model, "predict_proba"):
        return np.asarray(model.predict_proba(Xs), dtype=np.float64)
    return softmax(predict_logits(model, Xs))

def entropy_margin(probs: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    p = np.clip(probs, 1e-12, 1.0)
    ent = -(p * np.log(p)).sum(axis=1, keepdims=True)
    top2 = np.partition(p, -2, axis=1)[:, -2:]
    mar = (top2[:, 1] - top2[:, 0]).reshape(-1, 1)
    return ent, mar

def js_divergence(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    p = np.clip(p, 1e-12, 1.0); q = np.clip(q, 1e-12, 1.0)
    m = 0.5 * (p + q)
    kl_pm = (p * (np.log(p) - np.log(m))).sum(axis=1)
    kl_qm = (q * (np.log(q) - np.log(m))).sum(axis=1)
    return (0.5 * (kl_pm + kl_qm)).reshape(-1, 1)

def disagreement_count(prob_blocks: Dict[str, np.ndarray]) -> np.ndarray:
    preds = np.stack([np.argmax(v, axis=1) for v in prob_blocks.values()], axis=1)
    out = [((row != np.unique(row, return_counts=True)[0][np.argmax(np.unique(row, return_counts=True)[1])]).sum(),) for row in preds]
    return np.array(out, dtype=np.float64)

def temperature_from_logits(logits: np.ndarray, y_true: np.ndarray) -> float:
    def nll(T):
        return log_loss(y_true, softmax(logits / float(T[0])), labels=np.arange(n_classes))
    r = minimize(nll, x0=[1.0], bounds=[(0.2, 5.0)], method="L-BFGS-B")
    return float(np.clip(r.x[0], 0.2, 5.0))

def make_base(name: str):
    if name == "svm": return Pipeline([("scaler", StandardScaler()), ("clf", SVC(C=8.0, kernel="rbf", probability=True, random_state=rng))])
    if name == "rf": return RandomForestClassifier(n_estimators=600, n_jobs=-1, random_state=rng)
    if name == "xgb" and HAS_XGB: return XGBClassifier(objective="multi:softprob", num_class=n_classes, tree_method="hist", n_estimators=800, learning_rate=0.05, max_depth=6, subsample=0.9, colsample_bytree=0.9, random_state=rng, n_jobs=-1)
    if name == "knn": return Pipeline([("scaler", StandardScaler()), ("clf", KNeighborsClassifier(n_neighbors=5, weights="distance"))])
    if name == "cat" and HAS_CAT: return CatBoostClassifier(loss_function="MultiClass", iterations=800, depth=6, learning_rate=0.06, random_seed=rng, verbose=False)
    if name == "lgbm" and HAS_LGBM: return lgb.LGBMClassifier(objective="multiclass", num_class=n_classes, n_estimators=900, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9, random_state=rng, n_jobs=-1, verbose=-1)
    raise ValueError(f"Unknown or unavailable base model: {name}")

def try_load_or_build(path: str, name: str):
    if path and os.path.exists(path):
        try:
            model_obj = joblib.load(path) if not path.endswith('.cbm') else make_base(name).load_model(path)
            print(f"Successfully loaded '{name}' from file.")
            return model_obj
        except Exception as e:
            print(f"Failed to load '{name}' from {path} (Reason: {e}). Retraining from scratch.")
    return make_base(name)

base_names: List[str] = ["svm", "rf", "knn"]
if HAS_XGB: base_names.insert(1, "xgb")
if HAS_CAT: base_names.append("cat")
if HAS_LGBM: base_names.append("lgbm")

bases: Dict[str, object] = {}
for b_name in base_names:
    model_path = SAVED_MODELS.get(b_name, "")
    est = try_load_or_build(model_path, b_name)
    bases[b_name] = est["model"] if isinstance(est, dict) and "model" in est else est
print("Using base models:", list(bases.keys()))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=rng)
oof_probs = {b: np.zeros((len(y_trval), n_classes)) for b in bases}
oof_logits = {b: np.zeros((len(y_trval), n_classes)) for b in bases}

print("Starting Out-of-Fold predictions...")
for k, (tr_idx, va_idx) in enumerate(skf.split(X_trval, y_trval), 1):
    Xtr, Xva = X_trval[tr_idx], X_trval[va_idx]
    ytr = y_trval[tr_idx]
    for b_name, est_template in bases.items():
        est = make_base(b_name) 
        est.fit(Xtr, ytr)
        oof_probs[b_name][va_idx] = predict_proba_safe(est, Xva)
        oof_logits[b_name][va_idx] = predict_logits(est, Xva)
    print(f"Fold {k} complete.")

CALIBRATE = {"svm", "knn"}
temperatures = {b: temperature_from_logits(oof_logits[b], y_trval) if b in CALIBRATE else 1.0 for b in bases}
print("Optimal Temperatures:", temperatures)
oof_probs_cal = {b: softmax(oof_logits[b] / T) for b, T in temperatures.items()}

def make_meta_features(prob_blocks: Dict[str, np.ndarray]) -> np.ndarray:
    names = sorted(prob_blocks.keys())
    features = [prob_blocks[n] for n in names]
    for n in names:
        ent, mar = entropy_margin(prob_blocks[n])
        features.extend([ent, mar])
    features.append(disagreement_count(prob_blocks))
    for n1, n2 in combinations(names, 2):
        features.append(js_divergence(prob_blocks[n1], prob_blocks[n2]))
    return np.hstack(features)

X_meta_oof = make_meta_features(oof_probs_cal)
print(f"OOF Meta-feature shape: {X_meta_oof.shape}")

meta_learners = {
    "logreg": LogisticRegression(multi_class="multinomial", solver="lbfgs", C=1.0, random_state=rng, n_jobs=-1, max_iter=1000),
    "mlp": MLPClassifier(hidden_layer_sizes=(128, 64), activation="relu", solver="adam", random_state=rng, max_iter=500)
}
if HAS_XGB: meta_learners["xgb"] = XGBClassifier(objective="multi:softprob", num_class=n_classes, n_estimators=500, learning_rate=0.05, max_depth=4, random_state=rng, n_jobs=-1)

oof_meta_acc = {}
for name, model in meta_learners.items():
    model.fit(X_meta_oof, y_trval)
    oof_preds = model.predict(X_meta_oof)
    acc = accuracy_score(y_trval, oof_preds)
    oof_meta_acc[name] = acc
    print(f"OOF Meta-learner '{name}' accuracy: {acc:.4f}")
best_meta_name = max(oof_meta_acc, key=oof_meta_acc.get)
best_meta_model = meta_learners[best_meta_name]
print(f"Best OOF meta-learner: '{best_meta_name}'")

def nll_blend_loss(weights, prob_blocks, y_true):
    probs = sum(w * prob_blocks[n] for w, n in zip(weights, sorted(prob_blocks.keys())))
    return log_loss(y_true, probs, labels=np.arange(n_classes))

initial_weights = np.ones(len(bases)) / len(bases)
bounds = [(0, 1)] * len(bases)
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
res = minimize(nll_blend_loss, initial_weights, args=(oof_probs_cal, y_trval), method='SLSQP', bounds=bounds, constraints=constraints)
blend_weights = res.x
blend_names = sorted(bases.keys())
print("Optimized Blend Weights:", {n: f"{w:.4f}" for n, w in zip(blend_names, blend_weights)})



print("Refitting base models on full trainval set for test prediction...")
fitted_bases = {}
for b_name in bases.keys(): 
    print(f"  -> Fitting final '{b_name}' model...")

    final_model = make_base(b_name)
    final_model.fit(X_trval, y_trval)
    fitted_bases[b_name] = final_model

test_probs = {b: predict_proba_safe(fitted_bases[b], X_test) for b in bases}
test_logits = {b: predict_logits(fitted_bases[b], X_test) for b in bases}
test_probs_cal = {b: softmax(test_logits[b] / temperatures[b]) for b in bases}

X_meta_test = make_meta_features(test_probs_cal)

ptest_meta = best_meta_model.predict_proba(X_meta_test)
pred_meta = ptest_meta.argmax(1)
acc_meta = accuracy_score(y_test, pred_meta)
f1_meta = f1_score(y_test, pred_meta, average="macro")

ptest_blend = sum(w * test_probs_cal[n] for w, n in zip(blend_weights, blend_names))
pred_blend = ptest_blend.argmax(1)
acc_blend = accuracy_score(y_test, pred_blend)
f1_blend = f1_score(y_test, pred_blend, average="macro")

lams, best_lam, best_mix_acc = np.linspace(0, 1, 21), 0.0, -1.0
p_meta_oof_preds = best_meta_model.predict_proba(X_meta_oof)
p_blend_oof = sum(w * oof_probs_cal[n] for w, n in zip(blend_weights, blend_names))
for lam in lams:
    acc = accuracy_score(y_trval, (lam * p_meta_oof_preds + (1 - lam) * p_blend_oof).argmax(1))
    if acc > best_mix_acc: best_mix_acc, best_lam = acc, lam
ptest_mix = best_lam * ptest_meta + (1 - best_lam) * ptest_blend
pred_mix = ptest_mix.argmax(1)
acc_mix = accuracy_score(y_test, pred_mix)
f1_mix = f1_score(y_test, pred_mix, average="macro")

candidates = {
    best_meta_name: (acc_meta, f1_meta, pred_meta, ptest_meta),
    "blend": (acc_blend, f1_blend, pred_blend, ptest_blend),
    f"mix_{best_meta_name}_blend": (acc_mix, f1_mix, pred_mix, ptest_mix)
}
winner_name = max(candidates.items(), key=lambda kv: kv[1][0])[0]
w_acc, w_f1, w_pred, w_proba = candidates[winner_name]

print(f"\n--- TEST RESULTS ---")
print(f"{best_meta_name}: acc={acc_meta:.4f}, f1_macro={f1_meta:.4f}")
print(f"Blend: acc={acc_blend:.4f}, f1_macro={f1_blend:.4f}")
print(f"Mix: acc={acc_mix:.4f}, f1_macro={f1_mix:.4f} (λ={best_lam:.2f})")
print(f"\nWINNER: {winner_name} with Accuracy = {w_acc:.4f}, F1-Macro = {w_f1:.4f}")

rep = classification_report(y_test, w_pred, target_names=classes, output_dict=True)
pd.DataFrame(rep).transpose().to_csv(os.path.join(OUT_DIR, f"report_{winner_name}.csv"))

cm = confusion_matrix(y_test, w_pred)
plot_confusion_matrix(cm, classes, os.path.join(OUT_DIR, f"cm_{winner_name}.png"), f"Confusion Matrix - {winner_name}")

y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))
plot_roc_pr_curves(y_test_bin, w_proba, classes, OUT_DIR, winner_name)


summary = {
    "ensemble_version": "V3", "winner": winner_name,
    "test_metrics": {k: {"acc": float(v[0]), "f1_macro": float(v[1])} for k, v in candidates.items()},
    "temperatures": temperatures,
    "blend_weights": {n: w for n, w in zip(blend_names, blend_weights)},
    "mix_lambda": float(best_lam)
}
with open(os.path.join(OUT_DIR, "summary_v3.json"), "w") as f: json.dump(summary, f, indent=4)
joblib.dump(best_meta_model, os.path.join(OUT_DIR, f"meta_model_{best_meta_name}_v3.joblib"))
print(f"\nSaved all V3 artifacts to: {OUT_DIR}")

Zipped folder saved to: /kaggle/working/apt/ensemble_model.zip


#### Direct ML

In [ ]:

import os, json, glob, joblib, warnings
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

try:
    from catboost import CatBoostClassifier
    HAS_CAT = True
except Exception:
    HAS_CAT = False

warnings.filterwarnings("ignore")
RNG = 42

DATA_ROOT = "/kaggle/input/jackfruit/AugmentedJackfruit"   
OUT_DIR   = "/kaggle/working/ML_Direct"
os.makedirs(OUT_DIR, exist_ok=True)

IMG_SIZE = (64, 64)    
PCA_N    = 256         

def load_images_from_root(root, size=(64,64)):
    cls_dirs = sorted([d for d in glob.glob(os.path.join(root, "*")) if os.path.isdir(d)])
    if not cls_dirs:
        raise RuntimeError(f"No class folders under: {root}")
    X_list, y_list, classes = [], [], [os.path.basename(d) for d in cls_dirs]
    for ci, d in enumerate(cls_dirs):
        paths = []
        for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.tif","*.tiff","*.webp"):
            paths += glob.glob(os.path.join(d, ext))
        for p in paths:
            im = Image.open(p).convert("RGB").resize(size, Image.BILINEAR)
            X_list.append(np.asarray(im, dtype=np.uint8).reshape(-1))
            y_list.append(ci)
    X = np.stack(X_list).astype(np.float32) / 255.0
    y = np.array(y_list, dtype=np.int64)
    return X, y, classes

X_all, y_all, class_names = load_images_from_root(DATA_ROOT, IMG_SIZE)
n_classes = len(np.unique(y_all))
print(f"Loaded {X_all.shape[0]} images; raw dim={X_all.shape[1]}; classes={n_classes}")


X_trv, X_test, y_trv, y_test = train_test_split(X_all, y_all, test_size=0.20, stratify=y_all, random_state=RNG)
X_train, X_val, y_train, y_val = train_test_split(X_trv, y_trv, test_size=0.125, stratify=y_trv, random_state=RNG)
print({"train": len(y_train), "val": len(y_val), "test": len(y_test)})

scaler = StandardScaler(with_mean=True, with_std=True)
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

if PCA_N is not None:
    pca = PCA(n_components=PCA_N, random_state=RNG)
    X_train_t = pca.fit_transform(X_train_s)
    X_val_t   = pca.transform(X_val_s)
    X_test_t  = pca.transform(X_test_s)
    trans_desc = f"StandardScaler + PCA({PCA_N})"
else:
    pca = None
    X_train_t, X_val_t, X_test_t = X_train_s, X_val_s, X_test_s
    trans_desc = "StandardScaler only"

print("Transform:", trans_desc, "| dim:", X_train_t.shape[1])

def eval_and_save(name, model, Xt, yt, split, out_dir):
    y_pred = model.predict(Xt)
    acc = accuracy_score(yt, y_pred)
    f1m = f1_score(yt, y_pred, average="macro")
    rep = classification_report(yt, y_pred, target_names=class_names, output_dict=True)
    cm  = confusion_matrix(yt, y_pred)
    pd.DataFrame(rep).transpose().to_csv(os.path.join(out_dir, f"{name}_{split}_report.csv"))
    pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(os.path.join(out_dir, f"{name}_{split}_cm.csv"))

    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest'); plt.title(f"{name} — {split} CM"); plt.colorbar()
    ticks = np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right"); plt.yticks(ticks, class_names)
    th = cm.max()/2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i,j]), ha="center", va="center",
                     color="white" if cm[i,j] > th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(os.path.join(out_dir, f"{name}_{split}_cm.png"), dpi=160, bbox_inches="tight")
    plt.close()
    return {"accuracy": float(acc), "f1_macro": float(f1m)}

def save_bundle(name, model, out_dir, scaler, pca):
    os.makedirs(out_dir, exist_ok=True)
    if name.lower().startswith("catboost"):
        model.save_model(os.path.join(out_dir, f"{name}.cbm"))
        joblib.dump({"scaler": scaler, "pca": pca, "classes": class_names, "img_size": IMG_SIZE},
                    os.path.join(out_dir, f"{name}_preproc.joblib"))
    else:
        joblib.dump({"model": model, "scaler": scaler, "pca": pca, "classes": class_names, "img_size": IMG_SIZE},
                    os.path.join(out_dir, f"{name}.joblib"))

def train_and_report(name, model, Xt, yt, Xv, yv, Xte, yte):
    print(f"\n=== {name} ===")
    d = os.path.join(OUT_DIR, name.replace(" ", "_")); os.makedirs(d, exist_ok=True)
    model.fit(Xt, yt)
    val_sum  = eval_and_save(name, model, Xv,  yv,  "val",  d)
    test_sum = eval_and_save(name, model, Xte, yte, "test", d)
    save_bundle(name, model, d, scaler, pca)
    with open(os.path.join(d, "summary.json"), "w") as f:
        json.dump({"transform": trans_desc, "val": val_sum, "test": test_sum, "classes": class_names}, f, indent=2)
    print(f"{name} -> VAL acc {val_sum['accuracy']:.4f} | TEST acc {test_sum['accuracy']:.4f}")

lr  = LogisticRegression(C=1.0, penalty="l2", solver="lbfgs",
                         multi_class="multinomial", max_iter=1000, n_jobs=-1, random_state=RNG)
train_and_report("LR_multinomial", lr, X_train_t, y_train, X_val_t, y_val, X_test_t, y_test)

svm = SVC(kernel="rbf", C=10.0, gamma="scale", probability=True, random_state=RNG)
train_and_report("SVM_RBF", svm, X_train_t, y_train, X_val_t, y_val, X_test_t, y_test)

rf  = RandomForestClassifier(n_estimators=600, max_depth=None, n_jobs=-1, random_state=RNG)
train_and_report("RF", rf, X_train_t, y_train, X_val_t, y_val, X_test_t, y_test)

if HAS_XGB:
    xgb = XGBClassifier(
        objective="multi:softprob", num_class=n_classes, eval_metric="mlogloss",
        tree_method="hist", n_estimators=800, learning_rate=0.05, max_depth=6,
        subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0, reg_alpha=0.0,
        random_state=RNG
    )
    train_and_report("XGB", xgb, X_train_t, y_train, X_val_t, y_val, X_test_t, y_test)
else:
    print("XGBoost not available — skipping.")

knn = KNeighborsClassifier(n_neighbors=5, weights="distance", n_jobs=-1)
train_and_report("KNN", knn, X_train_t, y_train, X_val_t, y_val, X_test_t, y_test)

if HAS_CAT:
    cat = CatBoostClassifier(loss_function="MultiClass" if n_classes>2 else "Logloss",
                             eval_metric="MultiClass", iterations=600, depth=6, learning_rate=0.06,
                             random_seed=RNG, verbose=False)
    train_and_report("CatBoost", cat, X_train_t, y_train, X_val_t, y_val, X_test_t, y_test)
else:
    print("CatBoost not available — skipping.")

print("\nDone. Models & reports saved in:", OUT_DIR)


Loaded 9000 images; raw dim=12288; classes=6
{'train': 6300, 'val': 900, 'test': 1800}
Transform: StandardScaler + PCA(256) | dim: 256

=== LR_multinomial ===
LR_multinomial -> VAL acc 0.7800 | TEST acc 0.7833

=== SVM_RBF ===
SVM_RBF -> VAL acc 0.9478 | TEST acc 0.9422

=== RF ===
RF -> VAL acc 0.9056 | TEST acc 0.9039

=== XGB ===
XGB -> VAL acc 0.9111 | TEST acc 0.9156

=== KNN ===
KNN -> VAL acc 0.8956 | TEST acc 0.8800

=== CatBoost ===
CatBoost -> VAL acc 0.9100 | TEST acc 0.9044

Done. Models & reports saved in: /kaggle/working/ML_Direct
